<a href="https://colab.research.google.com/github/22130056-HoHoangDuy/Labs_ML/blob/main/22130056_HoHoangDuy_Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import
# Cài keras-tuner
!pip install -q -U keras-tuner

import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.8 MB/s eta 0:00:00


In [3]:
#Connect to Drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML_Labs_Datasets'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML_Labs_Datasets


In [4]:
#Task1.1
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(250, activation='relu'),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(x_train, y_train_cat, epochs=10, batch_size=128, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(x_test, y_test_cat)
print(f"Test accuracy: {test_accuracy:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8489 - loss: 0.5397 - val_accuracy: 0.9568 - val_loss: 0.1392
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9637 - loss: 0.1214 - val_accuracy: 0.9767 - val_loss: 0.0835
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9795 - loss: 0.0695 - val_accuracy: 0.9755 - val_loss: 0.0788
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9854 - loss: 0.0496 - val_accuracy: 0.9792 - val_loss: 0.0771
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9889 - loss: 0.0364 - val_accuracy: 0.9805 - val_loss: 0.0684
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9911 - loss: 0.0280 - val_accuracy: 0.9772 - val_loss: 0.0777
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9926 - loss: 0.0223 - val_accuracy: 0.9797 - val_loss: 0.0767
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9955 - loss: 0.0160 - val_accuracy

In [5]:
#Task1.2
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    optimizer = hp.Choice('optimizer', ['Adam', 'SGD', 'RMSprop'])

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

import keras_tuner as kt

tuner = kt.GridSearch(
    build_model,
    objective='val_accuracy',
    max_trials=27,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mnist_tuning'
)

tuner.search_space_summary()

tuner.search(
    x_train, y_train_cat,
    epochs=10,
    validation_split=0.1,
    callbacks=[],
    batch_size=128
)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Reloading Tuner from my_dir/mnist_tuning/tuner0.json
Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'Adam', 'conditions': [], 'values': ['Adam', 'SGD', 'RMSprop'], 'ordered': False}


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 250)            │       196,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        25,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,360 (868.59 KB)

 Trainable params: 222,360 (868.59 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#Task2.1
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

# Kiểm tra GPU
print("GPU available:", tf.config.list_physical_devices('GPU'))
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                  kernel_constraint=MaxNorm(3), input_shape=(32, 32, 3)),
    layers.Dropout(0.2),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                  kernel_constraint=MaxNorm(3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_constraint=MaxNorm(3)),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Callback dừng sớm nếu không cải thiện
early_stop = EarlyStopping(monitor='val_loss', patience=3)

model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stop])
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")


GPU available: []
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 249ms/step - accuracy: 0.2696 - loss: 1.9921 - val_accuracy: 0.4215 - val_loss: 1.6327
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 196s 240ms/step - accuracy: 0.4483 - loss: 1.5247 - val_accuracy: 0.5232 - val_loss: 1.3417
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 244ms/step - accuracy: 0.5259 - loss: 1.3222 - val_accuracy: 0.5609 - val_loss: 1.2266
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 246ms/step - accuracy: 0.5741 - loss: 1.1948 - val_accuracy: 0.5940 - val_loss: 1.1254
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 197s 239ms/step - accuracy: 0.6127 - loss: 1.0850 - val_accuracy: 0.6204 - val_loss: 1.0824
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 205s 244ms/step - accuracy: 0.6511 - loss: 0.9797 - val_accuracy: 0.6377 - val_loss: 1.0388
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 198s 239ms/step - accuracy: 0.6893 - loss: 0.8740 - val_accuracy: 0.6467 - val_loss: 1.0087
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 239ms/step - accuracy: 0.7143 -

In [7]:
#Task2.2
import keras_tuner as kt

def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                            kernel_constraint=MaxNorm(3), input_shape=(32, 32, 3)))
    model.add(layers.Dropout(hp.Float('dropout1', 0.1, 0.3, step=0.05)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                            kernel_constraint=MaxNorm(3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', 256, 512, step=128),
                           activation='relu', kernel_constraint=MaxNorm(3)))
    model.add(layers.Dropout(hp.Float('dropout2', 0.3, 0.6, step=0.1)))
    model.add(layers.Dense(10, activation='softmax'))

    opt = SGD(learning_rate=hp.Choice('learning_rate', [0.01, 0.005, 0.001]), momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cifar10_cnn'
)

tuner.search(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Lấy model tốt nhất
best_model = tuner.get_best_models(num_models=1)[0]

# Đánh giá model tốt nhất
loss, acc = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Tuned Model Accuracy: {acc:.4f}")


Trial 5 Complete [00h 24m 41s]
val_accuracy: 0.6765000224113464

Best val_accuracy So Far: 0.6765000224113464
Total elapsed time: 6d 01h 23m 13s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Tuned Model Accuracy: 0.6722


In [8]:
#Task3.1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers
df = pd.read_csv('creditcard.csv')
X = df.drop(columns=['Class'])
y = df['Class']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X_cnn, y, test_size=0.2, stratify=y, random_state=42)

model = models.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Dropout(0.3),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=2048, validation_split=0.2)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 11s 105ms/step - accuracy: 0.9565 - loss: 0.1723 - val_accuracy: 0.9982 - val_loss: 0.0269
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.9982 - loss: 0.0275 - val_accuracy: 0.9982 - val_loss: 0.0081
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - accuracy: 0.9984 - loss: 0.0096 - val_accuracy: 0.9994 - val_loss: 0.0047
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - accuracy: 0.9990 - loss: 0.0055 - val_accuracy: 0.9993 - val_loss: 0.0040
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - accuracy: 0.9991 - loss: 0.0043 - val_accuracy: 0.9994 - val_loss: 0.0038
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.9991 - loss: 0.0046 - val_accuracy: 0.9993 - val_loss: 0.0039
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.9992 - loss: 0.0044 - val_accuracy: 0.9994 - val_loss: 0.0037
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.9993 - loss: 0.0036 - val_accuracy: 0.9993 - val

In [9]:
#Task3.2
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam, RMSprop

def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=hp.Int('conv1_filters', 16, 64, step=16),
                            kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(hp.Float('dropout1', 0.2, 0.5, step=0.1)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout2', 0.3, 0.6, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='cc_fraud_dir',
    project_name='cnn_tuning'
)
tuner.search(X_train, y_train, epochs=10, batch_size=2048, validation_split=0.2)

best_model = tuner.get_best_models(1)[0]
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Best Tuned Model Accuracy: {acc:.4f}")

Trial 3 Complete [00h 00m 45s]
val_accuracy: 0.9993636012077332

Best val_accuracy So Far: 0.9993636012077332
Total elapsed time: 00h 02m 44s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9994 - loss: 0.0026
Best Tuned Model Accuracy: 0.9993


In [ ]:
#Task4.1
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train = X_train[..., tf.newaxis]
X_test  = X_test[..., tf.newaxis]

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 42s 95ms/step - accuracy: 0.6865 - loss: 2.6861 - val_accuracy: 0.8558 - val_loss: 0.3891
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 95ms/step - accuracy: 0.8723 - loss: 0.3546 - val_accuracy: 0.8728 - val_loss: 0.3426
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 96ms/step - accuracy: 0.8846 - loss: 0.3076 - val_accuracy: 0.8785 - val_loss: 0.3426
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 94ms/step - accuracy: 0.9016 - loss: 0.2666 - val_accuracy: 0.8777 - val_loss: 0.3140
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 93ms/step - accuracy: 0.9045 - loss: 0.2531 - val_accuracy: 0.8888 - val_loss: 0.3054
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 90ms/step - accuracy: 0.9165 - loss: 0.2250 - val_accuracy: 0.8850 - val_loss: 0.2948
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.9218 - loss: 0.2085 - val_accuracy: 0.8970 - val_loss: 0.3120
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 89ms/step - accuracy: 0.9286 - loss: 0.1918 - 

In [11]:
#Task4.2
import keras_tuner as kt
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(
        filters=hp.Int('conv_filters', 32, 64, step=16),
        kernel_size=3,
        activation='relu',
        input_shape=(28, 28, 1)
    ))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.Conv2D(64, 3, activation='relu'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(
        hp.Int('dense_units', 64, 128, step=32),
        activation='relu'
    ))
    model.add(layers.Dropout(hp.Float('dropout', 0.3, 0.6, step=0.1)))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='fashion_tuning',
    project_name='fashion_mnist_cnn'
)
tuner.search(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)
best_model = tuner.get_best_models(1)[0]
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Best Tuned Model Accuracy: {acc:.4f}")

Trial 3 Complete [00h 13m 04s]
val_accuracy: 0.9028333425521851

Best val_accuracy So Far: 0.9028333425521851
Total elapsed time: 00h 35m 00s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8974 - loss: 0.3094
Best Tuned Model Accuracy: 0.8964
